### BIG DATA FINAL PROJECT

In [1]:
!pip install pyspark
!pip install pymongo
!pip install pandas
!pip install geopy
!pip install folium
!pip install haversine

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

# create a SparkSession with increased memory allocation
conf = SparkConf() \
    .set("spark.driver.memory", "8g") \
    .set("spark.executor.memory", "8g") \
    .set("spark.network.timeout", "600s")

spark = SparkSession.builder \
    .appName("NYPD") \
    .config(conf=conf) \
    .getOrCreate()

# load the NYPD Complaints dataset into a Spark DataFrame
complaintsDF = spark.read.csv("NYPD_Complaint_Data_Historic.csv", header=True, inferSchema=True)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/09 01:09:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Only using half the dataset as its too big and taking too long and crashing sometimes on my PC
sampledDF = complaintsDF.sample(withReplacement=False, fraction=0.25, seed=42)

In [4]:
df = sampledDF

## DATA PROFILING

In [5]:
df.printSchema()

# Display the number of rows and columns in the dataframe
print("Number of rows: ", df.count())
print("Number of columns: ", len(df.columns))

df.show(5)

# Display summary statistics of the dataframe
df.describe().show()

root
 |-- CMPLNT_NUM: integer (nullable = true)
 |-- CMPLNT_FR_DT: string (nullable = true)
 |-- CMPLNT_FR_TM: timestamp (nullable = true)
 |-- CMPLNT_TO_DT: string (nullable = true)
 |-- CMPLNT_TO_TM: string (nullable = true)
 |-- ADDR_PCT_CD: integer (nullable = true)
 |-- RPT_DT: string (nullable = true)
 |-- KY_CD: integer (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- PD_CD: integer (nullable = true)
 |-- PD_DESC: string (nullable = true)
 |-- CRM_ATPT_CPTD_CD: string (nullable = true)
 |-- LAW_CAT_CD: string (nullable = true)
 |-- BORO_NM: string (nullable = true)
 |-- LOC_OF_OCCUR_DESC: string (nullable = true)
 |-- PREM_TYP_DESC: string (nullable = true)
 |-- JURIS_DESC: string (nullable = true)
 |-- JURISDICTION_CODE: integer (nullable = true)
 |-- PARKS_NM: string (nullable = true)
 |-- HADEVELOPT: string (nullable = true)
 |-- HOUSING_PSA: string (nullable = true)
 |-- X_COORD_CD: integer (nullable = true)
 |-- Y_COORD_CD: integer (nullable = true)
 |-- SUSP

Number of rows:  1958512
Number of columns:  35


23/05/08 16:28:13 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+------------+-------------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+-----------------+-------------+----------------+-----------------+--------+----------+-----------+----------+----------+--------------+---------+--------+----------------+-----------------+------------------+--------------------+--------------------+------------+-------------+--------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|       CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|      JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|         Latitude|         Longitude|             Lat_Lon|         PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|      VIC_RACE|VIC_SEX|
+----------+----------

+-------+-------------------+------------+------------+------------+------------------+----------+------------------+--------------------+------------------+--------------------+----------------+----------+-------------+-----------------+------------------+----------------+------------------+--------------------+--------------------+------------------+------------------+------------------+------------------+--------------------+--------+-----------------+-------------------+-------------------+--------------------+--------------------+--------------+-----------------+--------------------+-------+
|summary|         CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_TO_DT|CMPLNT_TO_TM|       ADDR_PCT_CD|    RPT_DT|             KY_CD|           OFNS_DESC|             PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|      BORO_NM|LOC_OF_OCCUR_DESC|     PREM_TYP_DESC|      JURIS_DESC| JURISDICTION_CODE|            PARKS_NM|          HADEVELOPT|       HOUSING_PSA|        X_COORD_CD|        Y_COORD_CD|    

In [5]:
from pyspark.sql.functions import col

# Generate column-level statistics using Spark DataFrame API
col_stats = df.select([col(c).cast("string") for c in df.columns]).describe()

# Display column-level statistics
col_stats.show()

# Generate row-level statistics using Spark DataFrame API
row_stats = df.summary()

row_stats.show()

23/05/09 01:11:01 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------------------+------------+-------------------+------------+------------+------------------+----------+------------------+--------------------+------------------+--------------------+----------------+----------+-------------+-----------------+------------------+----------------+------------------+--------------------+--------------------+------------------+------------------+------------------+------------------+--------------------+--------+-----------------+-------------------+-------------------+--------------------+--------------------+--------------+-----------------+--------------------+-------+
|summary|         CMPLNT_NUM|CMPLNT_FR_DT|       CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|       ADDR_PCT_CD|    RPT_DT|             KY_CD|           OFNS_DESC|             PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|      BORO_NM|LOC_OF_OCCUR_DESC|     PREM_TYP_DESC|      JURIS_DESC| JURISDICTION_CODE|            PARKS_NM|          HADEVELOPT|       HOUSING_PSA|  

+-------+-------------------+------------+------------+------------+------------------+----------+------------------+--------------------+------------------+--------------------+----------------+----------+-------------+-----------------+------------------+----------------+------------------+--------------------+--------------------+------------------+------------------+------------------+------------------+--------------------+--------+-----------------+-------------------+-------------------+--------------------+--------------------+--------------+-----------------+--------------------+-------+
|summary|         CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_TO_DT|CMPLNT_TO_TM|       ADDR_PCT_CD|    RPT_DT|             KY_CD|           OFNS_DESC|             PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|      BORO_NM|LOC_OF_OCCUR_DESC|     PREM_TYP_DESC|      JURIS_DESC| JURISDICTION_CODE|            PARKS_NM|          HADEVELOPT|       HOUSING_PSA|        X_COORD_CD|        Y_COORD_CD|    

## Cleaning, Validation

In [6]:
# check each col for percent missing value
from pyspark.sql.functions import count, when, col

# Convert the timestamp column to double
df = df.withColumn("CMPLNT_FR_TM_double", df["CMPLNT_FR_TM"].cast("double"))

# Calculate the count of missing values for each column
missing_values_count = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

# Calculate the percentage of missing values for each column
missing_values_percent = missing_values_count.select([(col(c) / df.count() * 100).alias(c + '_percent') for c in df.columns])

missing_values_percent.show()

+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+-------------+-------------------+------------------+------------------+------------------------+------------------+-------------------+-------------------------+---------------------+------------------+-------------------------+-----------------+------------------+-------------------+-------------------+-------------------+----------------------+-----------------+-----------------+------------------------+-------------------+-------------------+-------------------+-------------------+--------------------+---------------------+--------------------+--------------------+---------------------------+
|CMPLNT_NUM_percent|CMPLNT_FR_DT_percent|CMPLNT_FR_TM_percent|CMPLNT_TO_DT_percent|CMPLNT_TO_TM_percent| ADDR_PCT_CD_percent|RPT_DT_percent|KY_CD_percent|  OFNS_DESC_percent|     PD_CD_percent|   PD_DESC_percent|CRM_ATPT_CPTD_CD_percent|LAW_CAT_CD_percent|

In [7]:
# dropping rows with null values in selected columns
df = df.na.drop(subset=['Y_COORD_CD', 'X_COORD_CD', 'Latitude', 'Longitude', 'CRM_ATPT_CPTD_CD', 'CMPLNT_FR_TM', 'Lat_Lon', 'CMPLNT_FR_DT', 'BORO_NM', 'OFNS_DESC'])

In [8]:
# dropping columns that are not significant for future data exploration
df = df.drop('PARKS_NM', 'STATION_NAME', 'TRANSIT_DISTRICT', 'HADEVELOPT', 'HOUSING_PSA', 'CMPLNT_TO_DT', 'CMPLNT_TO_TM')

In [9]:
# replacing null values in 'LOC_OF_OCCUR_DESC' with 'UNKNOWN'
df = df.fillna({'LOC_OF_OCCUR_DESC':'UNKNOWN'})

In [10]:
# replacing null values in 'VIC_RACE' with 'UNKNOWN'
df = df.fillna({'VIC_RACE':'UNKNOWN'})

In [11]:
# replacing null values in 'VIC_AGE_GROUP' with 'UNKNOWN'
df = df.fillna({'VIC_AGE_GROUP':'UNKNOWN'})

In [12]:
# replacing null values in 'VIC_SEX' with 'UNKNOWN'
df = df.fillna({'VIC_SEX':'UNKNOWN'})

In [13]:
print('Clean dataset: ')
print("Observations: ", df.count())
print("Variables: ", len(df.columns))
df.show(5)

Clean dataset: 


Observations:  1946157
Variables:  29
+----------+------------+-------------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+-----------------+-------------+----------------+-----------------+----------+----------+--------------+---------+--------+-----------------+------------------+--------------------+--------------------+-------------+--------------+-------+-------------------+
|CMPLNT_NUM|CMPLNT_FR_DT|       CMPLNT_FR_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|      JURIS_DESC|JURISDICTION_CODE|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|         Latitude|         Longitude|             Lat_Lon|         PATROL_BORO|VIC_AGE_GROUP|      VIC_RACE|VIC_SEX|CMPLNT_FR_TM_double|
+----------+------------+-------------------+-----------+----------+-----+--------------------+-----+-------------------

In [14]:
df = df.fillna('UNKNOWN')

In [15]:
from pyspark.sql.types import StringType

for column in df.columns:
    if isinstance(df.schema[column].dataType, StringType):
        df.groupBy(column).count().orderBy(col('count').desc()).show(10)
        print('\n------------------------------------------\n')

+------------+-----+
|CMPLNT_FR_DT|count|
+------------+-----+
|  01/01/2010|  633|
|  01/01/2016|  599|
|  01/01/2011|  597|
|  01/01/2014|  596|
|  01/01/2008|  582|
|  01/01/2007|  574|
|  01/01/2017|  554|
|  01/01/2019|  542|
|  01/01/2012|  532|
|  01/01/2013|  522|
+------------+-----+
only showing top 10 rows


------------------------------------------



+----------+-----+
|    RPT_DT|count|
+----------+-----+
|11/01/2006|  489|
|05/29/2007|  487|
|06/05/2007|  483|
|06/06/2007|  468|
|09/17/2009|  465|
|06/21/2006|  458|
|08/16/2006|  458|
|07/23/2008|  456|
|10/04/2006|  456|
|10/23/2007|  456|
+----------+-----+
only showing top 10 rows


------------------------------------------



+--------------------+------+
|           OFNS_DESC| count|
+--------------------+------+
|       PETIT LARCENY|332434|
|       HARRASSMENT 2|254535|
|ASSAULT 3 & RELAT...|204862|
|CRIMINAL MISCHIEF...|197422|
|       GRAND LARCENY|169426|
|     DANGEROUS DRUGS|108730|
|OFF. AGNST PUB OR...| 99817|
|      FELONY ASSAULT| 77043|
|             ROBBERY| 70369|
|            BURGLARY| 66432|
+--------------------+------+
only showing top 10 rows


------------------------------------------



+--------------------+------+
|             PD_DESC| count|
+--------------------+------+
|           ASSAULT 3|168513|
|HARASSMENT,SUBD 3...|166889|
|LARCENY,PETIT FRO...|100945|
|AGGRAVATED HARASS...| 97233|
|HARASSMENT,SUBD 1...| 87646|
|LARCENY,PETIT FRO...| 76504|
|MISCHIEF, CRIMINA...| 67412|
|ASSAULT 2,1,UNCLA...| 62186|
|LARCENY,PETIT FRO...| 61673|
|CRIMINAL MISCHIEF...| 55329|
+--------------------+------+
only showing top 10 rows


------------------------------------------



+----------------+-------+
|CRM_ATPT_CPTD_CD|  count|
+----------------+-------+
|       COMPLETED|1913348|
|       ATTEMPTED|  32809|
+----------------+-------+


------------------------------------------



...

+--------------------+-----+
|             Lat_Lon|count|
+--------------------+-----+
|(40.750430768, -7...| 5204|
|(40.679700408, -7...| 3864|
|(40.791151867, -7...| 2141|
|(40.837323511, -7...| 1757|
|(40.710093847, -7...| 1637|
|(40.869058532, -7...| 1617|
|(40.733926841, -7...| 1614|
|(40.804384046, -7...| 1407|
|(40.756266207, -7...| 1405|
|(40.651700904, -7...| 1404|
+--------------------+-----+
only showing top 10 rows


------------------------------------------



+--------------------+------+
|         PATROL_BORO| count|
+--------------------+------+
|   PATROL BORO BRONX|421336|
|PATROL BORO BKLYN...|289722|
|PATROL BORO BKLYN...|287208|
|PATROL BORO MAN S...|235151|
|PATROL BORO MAN N...|232944|
|PATROL BORO QUEEN...|201867|
|PATROL BORO QUEEN...|187878|
|PATROL BORO STATE...| 90029|
|             UNKNOWN|    22|
+--------------------+------+


------------------------------------------



+-------------+------+
|VIC_AGE_GROUP| count|
+-------------+------+
|        25-44|646609|
|      UNKNOWN|606716|
|        45-64|338375|
|        18-24|196155|
|          <18| 88090|
|          65+| 70101|
|          929|     4|
|         -943|     3|
|          946|     3|
|          943|     3|
+-------------+------+
only showing top 10 rows


------------------------------------------



+--------------------+------+
|            VIC_RACE| count|
+--------------------+------+
|             UNKNOWN|636190|
|               BLACK|470018|
|               WHITE|332923|
|      WHITE HISPANIC|316066|
|ASIAN / PACIFIC I...|115400|
|      BLACK HISPANIC| 66872|
|AMERICAN INDIAN/A...|  8680|
|               OTHER|     8|
+--------------------+------+


------------------------------------------



+-------+------+
|VIC_SEX| count|
+-------+------+
|      F|763836|
|      M|644400|
|      E|294017|
|      D|243843|
|UNKNOWN|    61|
+-------+------+


------------------------------------------



In [16]:
from pyspark.sql.functions import when, col

# modify abbreviations in VIC_SEX and SUSP_SEX columns
df = df.withColumn('VIC_SEX', when(col('VIC_SEX') == 'U', 'UNKNOWN')
                          .when(col('VIC_SEX') == 'E', 'UNKNOWN')
                          .when(col('VIC_SEX') == 'D', 'BUSINESS/ORGANIZATION')
                          .when(col('VIC_SEX') == 'F', 'FEMALE')
                          .when(col('VIC_SEX') == 'M', 'MALE')
                          .otherwise(col('VIC_SEX')))
df = df.withColumn('SUSP_SEX', when(col('SUSP_SEX') == 'U', 'UNKNOWN')
                            .when(col('SUSP_SEX') == 'E', 'UNKNOWN')
                            .when(col('SUSP_SEX') == 'D', 'BUSINESS/ORGANIZATION')
                            .when(col('SUSP_SEX') == 'F', 'FEMALE')
                            .when(col('SUSP_SEX') == 'M', 'MALE')
                            .otherwise(col('SUSP_SEX')))

# remove incorrect age groups
age_categories = ['<18', '18-24', '25-44', '45-64', '65+', 'UNKNOWN']
df = df.filter(col('VIC_AGE_GROUP').isin(age_categories) & col('SUSP_AGE_GROUP').isin(age_categories))

In [17]:
df.show(10)

+----------+------------+-------------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+-----------------+-------------+----------------+-----------------+----------+----------+--------------+---------+--------+------------------+------------------+--------------------+--------------------+-------------+--------------+--------------------+-------------------+
|CMPLNT_NUM|CMPLNT_FR_DT|       CMPLNT_FR_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|      JURIS_DESC|JURISDICTION_CODE|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|          Latitude|         Longitude|             Lat_Lon|         PATROL_BORO|VIC_AGE_GROUP|      VIC_RACE|             VIC_SEX|CMPLNT_FR_TM_double|
+----------+------------+-------------------+-----------+----------+-----+--------------------+-----+--------------------+--------

In [18]:
df.count()

1945988

In [19]:
from pyspark.sql.functions import when, col

# Fill in missing values in the OFNS_DESC column with "OTHER"
df = df.withColumn('OFNS_DESC', when(col('OFNS_DESC').isNull(), 'OTHER')
                                .otherwise(col('OFNS_DESC')))

In [20]:
from pyspark.sql.functions import count

df.groupBy('OFNS_DESC').agg(count('*').alias('count')).show()

+--------------------+------+
|           OFNS_DESC| count|
+--------------------+------+
|OTHER TRAFFIC INF...|     4|
|ANTICIPATORY OFFE...|    34|
|   FELONY SEX CRIMES|     8|
|OTHER OFFENSES RE...|  4234|
|VEHICLE AND TRAFF...| 24955|
|KIDNAPPING & RELA...|   760|
|OFF. AGNST PUB OR...| 99801|
|PETIT LARCENY OF ...|   370|
|      FELONY ASSAULT| 77033|
|OFFENSES RELATED ...|   425|
|ALCOHOLIC BEVERAG...|   359|
|CRIMINAL MISCHIEF...|197409|
|         THEFT-FRAUD| 20064|
|   THEFT OF SERVICES|  1220|
|            JOSTLING|   109|
|MISCELLANEOUS PEN...| 49907|
|LOITERING/GAMBLIN...|    60|
|               ARSON|  4561|
|OFFENSES AGAINST ...|  4940|
|            GAMBLING|   797|
+--------------------+------+
only showing top 20 rows



In [21]:
from pyspark.sql.functions import when

# categorize property related offenses
prop = ['BURGLARY', 'PETIT LARCENY', 'GRAND LARCENY', 'ROBBERY', 'THEFT-FRAUD', 
        'GRAND LARCENY OF MOTOR VEHICLE', 'FORGERY', 'JOSTLING', 'ARSON',
        'PETIT LARCENY OF MOTOR VEHICLE', 'OTHER OFFENSES RELATED TO THEF',
        "BURGLAR'S TOOLS", 'FRAUDS', 'POSSESSION OF STOLEN PROPERTY',
        'CRIMINAL MISCHIEF & RELATED OF', 'OFFENSES INVOLVING FRAUD',
        'FRAUDULENT ACCOSTING', 'THEFT OF SERVICES']

# categorize sexual offenses
sexual = ['SEX CRIMES', 'HARRASSMENT 2', 'RAPE', 'PROSTITUTION & RELATED OFFENSES',
          'FELONY SEX CRIMES', 'LOITERING/DEVIATE SEX']

# categorize drugs/alcohol related offenses
drug_alch = ['DANGEROUS DRUGS', 'INTOXICATED & IMPAIRED DRIVING',
             'ALCOHOLIC BEVERAGE CONTROL LAW', 'INTOXICATED/IMPAIRED DRIVING',
             'UNDER THE INFLUENCE OF DRUGS', 'LOITERING FOR DRUG PURPOSES']

# categorize personal (assault/homicidal/kidnapping/weapon) offenses
personal = ['ASSAULT 3 & RELATED OFFENSES', 'FELONY ASSAULT',
            'OFFENSES AGAINST THE PERSON', 'HOMICIDE-NEGLIGENT,UNCLASSIFIE',
            'HOMICIDE-NEGLIGENT-VEHICLE', 'KIDNAPPING & RELATED OFFENSES',
            'ENDAN WELFARE INCOMP', 'OFFENSES RELATED TO CHILDREN',
            'CHILD ABANDONMENT/NON SUPPORT', 'KIDNAPPING', 'DANGEROUS WEAPONS',
            'UNLAWFUL POSS. WEAP. ON SCHOOL']

# categorize administrative/trespassing/loitering/traffic offenses
admin = ['OFF. AGNST PUB ORD SENSBLTY &', 'CRIMINAL TRESPASS', 
         'VEHICLE AND TRAFFIC LAWS', 'OFFENSES AGAINST PUBLIC ADMINI',
         'ADMINISTRATIVE CODE', 'OFFENSES AGAINST PUBLIC SAFETY',
         'LOITERING/GAMBLING (CARDS, DIC', 'DISORDERLY CONDUCT',
         'NEW YORK CITY HEALTH CODE', 'DISRUPTION OF A RELIGIOUS SERV',
         'LOITERING', 'ADMINISTRATIVE CODES']

# categorize remaining offenses as other
other = ['MISCELLANEOUS PENAL LAW', 'OFFENSES AGAINST MARRIAGE UNCL',
         'OTHER STATE LAWS (NON PENAL LAW)', 'FORTUNE TELLING',
         'NYS LAWS-UNCLASSIFIED VIOLATION', 'LOITERING/GAMBLING (CARDS, DIC',
         'GAMBLING', 'OTHER STATE LAWS (NON PENAL LA', 'OTHER STATE LAWS',
         'ANTICIPATORY OFFENSES', 'ESCAPE 3', 'AGRICULTURE & MRKTS LAW-UNCLASSIFIED',
         'NYS LAWS-UNCLASSIFIED FELONY', 'UNAUTHORIZED USE OF A VEHICLE', 'OTHER']


# combine all under a new column
offenses = ['PROPERTY', 'SEXUAL', 'DRUGS/ALCOHOL', 'PERSONAL', 'ADMINISTRATIVE']

conditions = [(df['OFNS_DESC'].isin(prop)),
              (df['OFNS_DESC'].isin(sexual)),
              (df['OFNS_DESC'].isin(drug_alch)),
              (df['OFNS_DESC'].isin(personal)),
              (df['OFNS_DESC'].isin(admin))]

df = df.withColumn('OFNS_CATS', when(conditions[0], offenses[0])
                   .when(conditions[1], offenses[1])
                   .when(conditions[2], offenses[2])
                   .when(conditions[3], offenses[3])
                   .when(conditions[4], offenses[4])
                   .otherwise('OTHER'))

In [22]:
df.show(10)

+----------+------------+-------------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+-----------------+-------------+----------------+-----------------+----------+----------+--------------+---------+--------+------------------+------------------+--------------------+--------------------+-------------+--------------+--------------------+-------------------+---------+
|CMPLNT_NUM|CMPLNT_FR_DT|       CMPLNT_FR_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|      JURIS_DESC|JURISDICTION_CODE|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|          Latitude|         Longitude|             Lat_Lon|         PATROL_BORO|VIC_AGE_GROUP|      VIC_RACE|             VIC_SEX|CMPLNT_FR_TM_double|OFNS_CATS|
+----------+------------+-------------------+-----------+----------+-----+--------------------+-----+---------

In [23]:
from pyspark.sql.functions import col

# fix datatypes of categorical columns
categorical_columns = set(df.columns) - set(['CMPLNT_NUM', 'RPT_DT', 'Latitude', 'Longitude', 'CMPLNT_DT'])
for column in categorical_columns:
    df = df.withColumn(column, col(column).cast('string'))

# set the order of some categorical columns so the visualizations in EDA will be consistent
df = df.withColumn('LAW_CAT_CD', df['LAW_CAT_CD'].cast('string').cast('string')
                   .cast("string")).orderBy(df['LAW_CAT_CD'].cast("string").isin(['VIOLATION', 'MISDEMEANOR', 'FELONY']).desc())
df = df.withColumn('SUSP_SEX', df['SUSP_SEX'].cast('string').cast('string')
                   .cast("string")).orderBy(df['SUSP_SEX'].cast("string").isin(['FEMALE', 'MALE', 'UNKNOWN']).desc())
df = df.withColumn('VIC_SEX', df['VIC_SEX'].cast('string').cast('string')
                   .cast("string")).orderBy(df['VIC_SEX'].cast("string").isin(['FEMALE', 'MALE', 'BUSINESS/ORGANIZATION', 'UNKNOWN']).desc())
df = df.withColumn('VIC_AGE_GROUP', df['VIC_AGE_GROUP'].cast('string').cast('string')
                   .cast("string")).orderBy(df['VIC_AGE_GROUP'].cast("string").isin(['<18', '18-24', '25-44', '45-64', '65+', 'UNKNOWN']).desc())
df = df.withColumn('SUSP_AGE_GROUP', df['SUSP_AGE_GROUP'].cast('string').cast('string')
                   .cast("string")).orderBy(df['SUSP_AGE_GROUP'].cast("string").isin(['<18', '18-24', '25-44', '45-64', '65+', 'UNKNOWN']).desc())
df = df.withColumn('OFNS_CATS', df['OFNS_CATS'].cast('string').cast('string')
                   .cast("string")).orderBy(df['OFNS_CATS'].cast("string").isin(['PROPERTY', 'PERSONAL', 'SEXUAL', 'ADMINISTRATIVE', 'DRUGS/ALCOHOL', 'OTHER']).desc())

In [24]:
from pyspark.sql.functions import col, to_date, to_timestamp

# Convert string columns to date and timestamp types
df = df.withColumn('CMPLNT_FR_DT', to_date(col('CMPLNT_FR_DT'), 'MM/dd/yyyy'))
df = df.withColumn('RPT_DT', to_date(col('RPT_DT'), 'MM/dd/yyyy'))


# Convert string columns to integer types
int_cols = ['ADDR_PCT_CD', 'KY_CD', 'PD_CD', 'JURISDICTION_CODE', 'X_COORD_CD', 'Y_COORD_CD']
for col_name in int_cols:
    df = df.withColumn(col_name, col(col_name).cast('integer'))

In [25]:
df = df.drop("JURISDICTION_CODE", "JURIS_DESC")

In [26]:
df.printSchema()

root
 |-- CMPLNT_NUM: integer (nullable = true)
 |-- CMPLNT_FR_DT: date (nullable = true)
 |-- CMPLNT_FR_TM: string (nullable = true)
 |-- ADDR_PCT_CD: integer (nullable = true)
 |-- RPT_DT: date (nullable = true)
 |-- KY_CD: integer (nullable = true)
 |-- OFNS_DESC: string (nullable = false)
 |-- PD_CD: integer (nullable = true)
 |-- PD_DESC: string (nullable = false)
 |-- CRM_ATPT_CPTD_CD: string (nullable = false)
 |-- LAW_CAT_CD: string (nullable = false)
 |-- BORO_NM: string (nullable = false)
 |-- LOC_OF_OCCUR_DESC: string (nullable = false)
 |-- PREM_TYP_DESC: string (nullable = false)
 |-- X_COORD_CD: integer (nullable = true)
 |-- Y_COORD_CD: integer (nullable = true)
 |-- SUSP_AGE_GROUP: string (nullable = false)
 |-- SUSP_RACE: string (nullable = false)
 |-- SUSP_SEX: string (nullable = false)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Lat_Lon: string (nullable = false)
 |-- PATROL_BORO: string (nullable = false)
 |-- VIC_AGE_GROUP:

## Merge with nyc neighbourhoods( add neighbourhood col in original dataset )

In [26]:
import requests
url = 'https://cocl.us/new_york_dataset'
response = requests.get(url)
newyork_data = response.json()

# extract neighborhoods data
neighborhoods_data = newyork_data['features']

# create a list of dictionaries to store the data
neighborhoods_list = []

# populate list with neighborhoods data
for data in neighborhoods_data:
    borough = data['properties']['borough']
    neighborhood_name = data['properties']['name']
    neighborhood_lat = data['geometry']['coordinates'][1]
    neighborhood_lon = data['geometry']['coordinates'][0]
    neighborhoods_list.append({'Borough': borough,
                               'Neighborhood': neighborhood_name,
                               'Latitude': neighborhood_lat,
                               'Longitude': neighborhood_lon})

# create a DataFrame from the list of dictionaries
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
schema = StructType([
    StructField('Borough', StringType(), True),
    StructField('Neighborhood', StringType(), True),
    StructField('Latitude', DoubleType(), True),
    StructField('Longitude', DoubleType(), True)
])
neighborhoodsDF = spark.createDataFrame(neighborhoods_list, schema=schema)

df.show()
neighborhoodsDF.show()

+----------+------------+-------------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+----------+----------+--------------+--------------+--------+------------------+------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------+
|CMPLNT_NUM|CMPLNT_FR_DT|       CMPLNT_FR_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|     SUSP_RACE|SUSP_SEX|          Latitude|         Longitude|             Lat_Lon|         PATROL_BORO|VIC_AGE_GROUP|            VIC_RACE|             VIC_SEX|     OFNS_CATS|
+----------+------------+-------------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+-------------

+---------+------------------+------------------+------------------+
|  Borough|      Neighborhood|          Latitude|         Longitude|
+---------+------------------+------------------+------------------+
|    Bronx|         Wakefield|    40.89470517661|-73.84720052054902|
|    Bronx|        Co-op City| 40.87429419303012|-73.82993910812398|
|    Bronx|       Eastchester|40.887555677350775|-73.82780644716412|
|    Bronx|         Fieldston| 40.89543742690383|-73.90564259591682|
|    Bronx|         Riverdale|40.890834493891305| -73.9125854610857|
|    Bronx|       Kingsbridge| 40.88168737120521|-73.90281798724604|
|Manhattan|       Marble Hill| 40.87655077879964|-73.91065965862981|
|    Bronx|          Woodlawn| 40.89827261213805|-73.86731496814176|
|    Bronx|           Norwood| 40.87722415599446| -73.8793907395681|
|    Bronx|    Williamsbridge| 40.88103887819211|-73.85744642974207|
|    Bronx|        Baychester|40.866858107252696|-73.83579759808117|
|    Bronx|    Pelham Parkway| 40.

In [27]:
from pyspark.sql.functions import col, lower

neighborCrimeDF = df.join(
    neighborhoodsDF.select([
        "Borough",
        "Neighborhood",
        col("Latitude").alias("NLatitude"),
        col("Longitude").alias("NLongitude")
    ]),
    lower(df.BORO_NM) == lower(neighborhoodsDF.Borough),
    "left"
)
neighborCrimeDF.show()

+----------+------------+-------------------+-----------+----------+-----+-----------------+-----+--------------------+----------------+----------+-------+-----------------+-------------+----------+----------+--------------+---------+--------+-----------+-------------+--------------------+-----------------+-------------+--------+-------+---------+-------+-------------------+------------------+------------------+
|CMPLNT_NUM|CMPLNT_FR_DT|       CMPLNT_FR_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|        OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|   Latitude|    Longitude|             Lat_Lon|      PATROL_BORO|VIC_AGE_GROUP|VIC_RACE|VIC_SEX|OFNS_CATS|Borough|       Neighborhood|         NLatitude|        NLongitude|
+----------+------------+-------------------+-----------+----------+-----+-----------------+-----+--------------------+----------------+----------+-------+-------------

In [28]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType
from haversine import haversine
from pyspark.sql.window import Window
from pyspark.sql.functions import udf

# define UDFs to calculate distance using haversine formula
udf_haversine = udf(lambda x, y: haversine(x, y, unit='mi') if x is not None and y is not None else None, DoubleType())
udf_string2array = udf(lambda lat, long: [float(lat), float(long)] if lat is not None and long is not None else None, ArrayType(DoubleType()))

# create new columns with latitude and longitude as arrays, and distance between crime location and neighborhood
neighborCrimeDF2 = neighborCrimeDF \
    .withColumn("LatLong", udf_string2array("Latitude", "Longitude")) \
    .withColumn("NLatLong", udf_string2array("NLatitude", "NLongitude")) \
    .withColumn("Distance", udf_haversine("LatLong", "NLatLong"))

# use window function to get the nearest neighborhood for each crime
window = Window.partitionBy("CMPLNT_NUM").orderBy(F.col("Distance").asc())
NCDF = neighborCrimeDF2 \
    .withColumn("DistanceRank", F.rank().over(window)) \
    .filter(F.col("DistanceRank") == 1)

## Store cleaned data to mongoDB database for analysis

In [32]:
rows = NCDF.rdd.map(lambda row: row.asDict()).collect()

In [40]:
for row in rows:
    row['RPT_DT'] = row['RPT_DT'].isoformat()        

In [33]:
from pyspark.sql import SparkSession
import pymongo
from bson import json_util

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["NYPD"]
collection = db["NYPD_Cleaned"]

In [41]:
collection.insert_many(rows)

### DATA ANALYSIS

In [52]:
count = collection.count_documents({})
print("Total count of documents in the collection:", count)

Total count of documents in the collection: 1945750


In [36]:
import pprint
pprint.pprint(rows[0], depth=1, width=80)

{'ADDR_PCT_CD': 108,
 'BORO_NM': 'QUEENS',
 'Borough': 'Queens',
 'CMPLNT_FR_DT': datetime.date(2014, 1, 14),
 'CMPLNT_FR_TM': '2023-05-08 00:25:00',
 'CMPLNT_NUM': 100208792,
 'CRM_ATPT_CPTD_CD': 'COMPLETED',
 'Distance': 0.15462914324868482,
 'DistanceRank': 1,
 'KY_CD': 351,
 'LAW_CAT_CD': 'MISDEMEANOR',
 'LOC_OF_OCCUR_DESC': 'INSIDE',
 'LatLong': [...],
 'Lat_Lon': '(40.748580841, -73.902061752)',
 'Latitude': 40.748580841,
 'Longitude': -73.902061752,
 'NLatLong': [...],
 'NLatitude': 40.74634908860222,
 'NLongitude': -73.90184166838284,
 'Neighborhood': 'Woodside',
 'OFNS_CATS': 'PROPERTY',
 'OFNS_DESC': 'CRIMINAL MISCHIEF & RELATED OF',
 'PATROL_BORO': 'PATROL BORO QUEENS NORTH',
 'PD_CD': 259,
 'PD_DESC': 'CRIMINAL MISCHIEF,UNCLASSIFIED 4',
 'PREM_TYP_DESC': 'RESIDENCE - APT. HOUSE',
 'RPT_DT': datetime.date(2014, 1, 14),
 'SUSP_AGE_GROUP': 'UNKNOWN',
 'SUSP_RACE': 'WHITE HISPANIC',
 'SUSP_SEX': 'MALE',
 'VIC_AGE_GROUP': '25-44',
 'VIC_RACE': 'WHITE HISPANIC',
 'VIC_SEX': 'MALE

In [53]:
first_doc = collection.find_one()
print(first_doc)

{'_id': ObjectId('6459872efa42a60e401075f4'), 'CMPLNT_NUM': 100208792, 'CMPLNT_FR_DT': '2014-01-14', 'CMPLNT_FR_TM': '2023-05-08 00:25:00', 'ADDR_PCT_CD': 108, 'RPT_DT': '2014-01-14', 'KY_CD': 351, 'OFNS_DESC': 'CRIMINAL MISCHIEF & RELATED OF', 'PD_CD': 259, 'PD_DESC': 'CRIMINAL MISCHIEF,UNCLASSIFIED 4', 'CRM_ATPT_CPTD_CD': 'COMPLETED', 'LAW_CAT_CD': 'MISDEMEANOR', 'BORO_NM': 'QUEENS', 'LOC_OF_OCCUR_DESC': 'INSIDE', 'PREM_TYP_DESC': 'RESIDENCE - APT. HOUSE', 'X_COORD_CD': 1011387, 'Y_COORD_CD': 212017, 'SUSP_AGE_GROUP': 'UNKNOWN', 'SUSP_RACE': 'WHITE HISPANIC', 'SUSP_SEX': 'MALE', 'Latitude': 40.748580841, 'Longitude': -73.902061752, 'Lat_Lon': '(40.748580841, -73.902061752)', 'PATROL_BORO': 'PATROL BORO QUEENS NORTH', 'VIC_AGE_GROUP': '25-44', 'VIC_RACE': 'WHITE HISPANIC', 'VIC_SEX': 'MALE', 'OFNS_CATS': 'PROPERTY', 'Borough': 'Queens', 'Neighborhood': 'Woodside', 'NLatitude': 40.74634908860222, 'NLongitude': -73.90184166838284, 'LatLong': [40.748580841, -73.902061752], 'NLatLong': [4

### How many crimes were reported in each borough?

In [54]:
result = collection.aggregate([
    {"$group": {"_id": "$BORO_NM", "count": {"$sum": 1}}}
])

for document in result:
    print(document)

{'_id': 'BROOKLYN', 'count': 576764}
{'_id': 'MANHATTAN', 'count': 468537}
{'_id': 'QUEENS', 'count': 389085}
{'_id': 'STATEN ISLAND', 'count': 90033}
{'_id': 'BRONX', 'count': 421331}


### To analyze the number of crimes reported in different boroughs over time:

In [56]:
result = collection.aggregate([
    {"$group": {"_id": {"borough": "$BORO_NM", "year": {"$year": {"$dateFromString": {"dateString": "$CMPLNT_FR_DT"}}}}, "count": {"$sum": 1}}}
])

for document in result:
    print(document)

{'_id': {'borough': 'MANHATTAN', 'year': 2011}, 'count': 28865}
{'_id': {'borough': 'STATEN ISLAND', 'year': 2009}, 'count': 6035}
{'_id': {'borough': 'BROOKLYN', 'year': 1919}, 'count': 1}
{'_id': {'borough': 'BRONX', 'year': 2018}, 'count': 25396}
{'_id': {'borough': 'BROOKLYN', 'year': 1955}, 'count': 1}
{'_id': {'borough': 'QUEENS', 'year': 2016}, 'count': 23784}
{'_id': {'borough': 'STATEN ISLAND', 'year': 1978}, 'count': 1}
{'_id': {'borough': 'MANHATTAN', 'year': 1912}, 'count': 2}
{'_id': {'borough': 'BROOKLYN', 'year': 1969}, 'count': 2}
{'_id': {'borough': 'BROOKLYN', 'year': 1993}, 'count': 7}
{'_id': {'borough': 'QUEENS', 'year': 2007}, 'count': 26326}
{'_id': {'borough': 'BROOKLYN', 'year': 1962}, 'count': 1}
{'_id': {'borough': 'MANHATTAN', 'year': 2015}, 'count': 28060}
{'_id': {'borough': 'STATEN ISLAND', 'year': 2002}, 'count': 6}
{'_id': {'borough': 'MANHATTAN', 'year': 2008}, 'count': 31902}
{'_id': {'borough': 'BRONX', 'year': 1968}, 'count': 2}
{'_id': {'borough': 

### To analyze trends in the types of crimes reported and the level of offense:

In [57]:
result = collection.aggregate([
    {"$group": {"_id": {"offense": "$OFNS_DESC", "level": "$LAW_CAT_CD"}, "count": {"$sum": 1}}},
    {"$sort": {"_id.offense": 1}}
])

for document in result:
    print(document)

{'_id': {'offense': 'ADMINISTRATIVE CODE', 'level': 'VIOLATION'}, 'count': 461}
{'_id': {'offense': 'ADMINISTRATIVE CODE', 'level': 'MISDEMEANOR'}, 'count': 3834}
{'_id': {'offense': 'ADMINISTRATIVE CODES', 'level': 'VIOLATION'}, 'count': 2}
{'_id': {'offense': 'AGRICULTURE & MRKTS LAW-UNCLASSIFIED', 'level': 'MISDEMEANOR'}, 'count': 199}
{'_id': {'offense': 'ALCOHOLIC BEVERAGE CONTROL LAW', 'level': 'MISDEMEANOR'}, 'count': 359}
{'_id': {'offense': 'ANTICIPATORY OFFENSES', 'level': 'MISDEMEANOR'}, 'count': 34}
{'_id': {'offense': 'ARSON', 'level': 'FELONY'}, 'count': 4561}
{'_id': {'offense': 'ASSAULT 3 & RELATED OFFENSES', 'level': 'MISDEMEANOR'}, 'count': 204821}
{'_id': {'offense': "BURGLAR'S TOOLS", 'level': 'MISDEMEANOR'}, 'count': 985}
{'_id': {'offense': 'BURGLARY', 'level': 'FELONY'}, 'count': 66423}
{'_id': {'offense': 'CHILD ABANDONMENT/NON SUPPORT', 'level': 'FELONY'}, 'count': 113}
{'_id': {'offense': 'CRIMINAL MISCHIEF & RELATED OF', 'level': 'FELONY'}, 'count': 34608}
{'

### To map the locations of crimes and identify areas with high crime rates:

In [58]:
from bson.son import SON

result = collection.aggregate([
    {"$group": {"_id": {"latitude": "$Latitude", "longitude": "$Longitude"}, "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1), ("_id", -1)])},
    {"$limit": 10}
])

for document in result:
    print(document)

{'_id': {'latitude': 40.750430768, 'longitude': -73.989282176}, 'count': 4903}
{'_id': {'latitude': 40.679700408, 'longitude': -73.776047368}, 'count': 3836}
{'_id': {'latitude': 40.791151867, 'longitude': -73.884371919}, 'count': 1950}
{'_id': {'latitude': 40.837323511, 'longitude': -73.919830757}, 'count': 1715}
{'_id': {'latitude': 40.869058532, 'longitude': -73.879630148}, 'count': 1580}
{'_id': {'latitude': 40.710093847, 'longitude': -74.01060963}, 'count': 1550}
{'_id': {'latitude': 40.733926841, 'longitude': -73.871582398}, 'count': 1419}
{'_id': {'latitude': 40.804384046, 'longitude': -73.937421669}, 'count': 1324}
{'_id': {'latitude': 40.75043076800005, 'longitude': -73.98928217599996}, 'count': 1282}
{'_id': {'latitude': 40.671106911, 'longitude': -73.881432957}, 'count': 1281}


### To identify the types of locations where crimes tend to occur:

In [59]:
result = collection.aggregate([
    {"$group": {"_id": "$PREM_TYP_DESC", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
])

for document in result:
    print(document)

{'_id': 'STREET', 'count': 618333}
{'_id': 'RESIDENCE - APT. HOUSE', 'count': 415758}
{'_id': 'RESIDENCE-HOUSE', 'count': 192167}
{'_id': 'RESIDENCE - PUBLIC HOUSING', 'count': 146440}
{'_id': 'OTHER', 'count': 51244}
{'_id': 'COMMERCIAL BUILDING', 'count': 49867}
{'_id': 'CHAIN STORE', 'count': 47085}
{'_id': 'TRANSIT - NYC SUBWAY', 'count': 41449}
{'_id': 'DEPARTMENT STORE', 'count': 39132}
{'_id': 'GROCERY/BODEGA', 'count': 25033}
{'_id': 'RESTAURANT/DINER', 'count': 22586}
{'_id': 'PARK/PLAYGROUND', 'count': 21788}
{'_id': 'PUBLIC SCHOOL', 'count': 21725}
{'_id': 'BAR/NIGHT CLUB', 'count': 20263}
{'_id': 'DRUG STORE', 'count': 20096}
{'_id': 'CLOTHING/BOUTIQUE', 'count': 16292}
{'_id': 'PUBLIC BUILDING', 'count': 11049}
{'_id': 'FOOD SUPERMARKET', 'count': 10497}
{'_id': 'PARKING LOT/GARAGE (PUBLIC)', 'count': 10401}
{'_id': 'PARKING LOT/GARAGE (PRIVATE)', 'count': 10085}
{'_id': 'FAST FOOD', 'count': 9644}
{'_id': 'HOSPITAL', 'count': 9279}
{'_id': 'HOTEL/MOTEL', 'count': 9141}
{'

### To identify patterns in suspect and victim characteristics

In [66]:
result = collection.aggregate([
    {"$group": {"_id": {"age": "$SUSP_AGE_GROUP", "race": "$SUSP_RACE", "sex": "$SUSP_SEX"}, "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
])

for document in result:
    print(document)

{'_id': {'age': 'UNKNOWN', 'race': 'UNKNOWN', 'sex': 'UNKNOWN'}, 'count': 1070367}
{'_id': {'age': 'UNKNOWN', 'race': 'BLACK', 'sex': 'MALE'}, 'count': 109623}
{'_id': {'age': '25-44', 'race': 'BLACK', 'sex': 'MALE'}, 'count': 98356}
{'_id': {'age': '25-44', 'race': 'WHITE HISPANIC', 'sex': 'MALE'}, 'count': 53236}
{'_id': {'age': 'UNKNOWN', 'race': 'UNKNOWN', 'sex': 'MALE'}, 'count': 48446}
{'_id': {'age': 'UNKNOWN', 'race': 'WHITE HISPANIC', 'sex': 'MALE'}, 'count': 40663}
{'_id': {'age': '18-24', 'race': 'BLACK', 'sex': 'MALE'}, 'count': 37830}
{'_id': {'age': 'UNKNOWN', 'race': 'BLACK', 'sex': 'FEMALE'}, 'count': 37509}
{'_id': {'age': '45-64', 'race': 'BLACK', 'sex': 'MALE'}, 'count': 34635}
{'_id': {'age': 'UNKNOWN', 'race': 'WHITE', 'sex': 'MALE'}, 'count': 31962}
{'_id': {'age': '25-44', 'race': 'WHITE', 'sex': 'MALE'}, 'count': 28331}
{'_id': {'age': '25-44', 'race': 'BLACK', 'sex': 'FEMALE'}, 'count': 28170}
{'_id': {'age': '18-24', 'race': 'WHITE HISPANIC', 'sex': 'MALE'}, '

### To identify clearance rates for different types of crimes, precincts, or boroughs:

In [67]:
result = collection.aggregate([
    {"$match": {"CRM_ATPT_CPTD_CD": "COMPLETED"}},
    {"$group": {"_id": {"offense": "$OFNS_DESC", "borough": "$BORO_NM", "year": {"$year": {"$dateFromString": {"dateString": "$CMPLNT_FR_DT"}}}}, "count": {"$sum": 1}}},
    {"$sort": {"_id.offense": 1}}
])

for document in result:
    print(document)

{'_id': {'offense': 'ADMINISTRATIVE CODE', 'borough': 'BRONX', 'year': 1998}, 'count': 1}
{'_id': {'offense': 'ADMINISTRATIVE CODE', 'borough': 'QUEENS', 'year': 2007}, 'count': 75}
{'_id': {'offense': 'ADMINISTRATIVE CODE', 'borough': 'QUEENS', 'year': 2005}, 'count': 1}
{'_id': {'offense': 'ADMINISTRATIVE CODE', 'borough': 'QUEENS', 'year': 2009}, 'count': 78}
{'_id': {'offense': 'ADMINISTRATIVE CODE', 'borough': 'QUEENS', 'year': 2015}, 'count': 77}
{'_id': {'offense': 'ADMINISTRATIVE CODE', 'borough': 'STATEN ISLAND', 'year': 2012}, 'count': 16}
{'_id': {'offense': 'ADMINISTRATIVE CODE', 'borough': 'QUEENS', 'year': 2013}, 'count': 74}
{'_id': {'offense': 'ADMINISTRATIVE CODE', 'borough': 'STATEN ISLAND', 'year': 2006}, 'count': 11}
{'_id': {'offense': 'ADMINISTRATIVE CODE', 'borough': 'BRONX', 'year': 2014}, 'count': 48}
{'_id': {'offense': 'ADMINISTRATIVE CODE', 'borough': 'STATEN ISLAND', 'year': 2008}, 'count': 16}
{'_id': {'offense': 'ADMINISTRATIVE CODE', 'borough': 'BROOKLYN